In [36]:
%pip install --upgrade scikit-image torch torchvision torchaudio kornia pytorch-lightning lightning wandb>=0.12.10 diffusers["torch"] transformers einops matplotlib requests click ema_pytorch pytorch_fid

Note: you may need to restart the kernel to use updated packages.


In [40]:
%%sh
cat scripts/train_cls_free.sh 
./scripts/train_cls_free.sh

nohup accelerate launch train_cls_free.py \
--train_data_path "LDM4DD/cifar100_train" \
--val_data_path "LDM4DD/cifar100_validation" \
--save_dir "saved_model/" \
--num_steps 50000 \
--num_timesteps 1000 \
--lr 1e-4 \
--save_freq 5000 \
> logs/train_cls_free_S50000_T1000.log 2>&1 &

In [2]:
from datasets.cifar_custom import __CIFAR_Customized, CIFAR10_Customized, CIFAR100_Customized
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--crop_size", type=int, default=256)
parser.add_argument("--train_data_path", type=str, default="cifar10_train")
parser.add_argument("--val_data_path", type=str, default="cifar10_validation")
parser.add_argument("--save_path", type=str, default="models")
parser.add_argument("--save_freq", type=int, default=10)
parser.add_argument("--top_k", type=int, default=1)
parser.add_argument("--lr", type=float, default=0.01)
parser.add_argument("--batch_size", type=int, default=16)
parser.add_argument("--num_epochs", type=int, default=20)
parser.add_argument("--max_steps", type=int, default=1000)
parser.add_argument("--log_steps", type=int, default=100)
parser.add_argument("--num_timesteps", type=int, default=100)
parser.add_argument("--warmup_steps", type=int, default=100)
parser.add_argument("--accelerator", type=str, default="cpu")
#parser.add_argument("--devices", type=str, default=1)

args = parser.parse_args(args=[])

In [3]:
import argparse
import pytorch_lightning as pl
import kornia.augmentation as KA
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from diffusion.LatentDiffusion import LatentDiffusionConditional
from diffusion.LatentDiffusion import LatentDiffusion
from datasets.example_dataset import ExampleImageDataset
from utils.EMA import EMA


def train(args):
    transform = [
        KA.RandomCrop((2 * args.crop_size, 2 * args.crop_size)),
        KA.Resize((args.crop_size, args.crop_size), antialias=True),
        KA.RandomVerticalFlip()
    ]

    train_ds = CIFAR100_Customized(args.train_data_path, train=True, conditional=False)
    val_ds = CIFAR100_Customized(args.val_data_path, train=False, conditional=False)

    model = LatentDiffusion(train_dataset=train_ds,
                                       valid_dataset=val_ds,
                                       num_timesteps=args.num_timesteps,
                                       lr=args.lr,
                                       num_warmup_steps=args.warmup_steps,
                                       num_epochs=args.num_epochs,
                                       batch_size=args.batch_size)

    wandb_logger = WandbLogger(project="CS5340")
    wandb_logger.experiment.config["key"] = "33da90c88f8e092fbf39fa7ad18e3125504b51a6"

    trainer = pl.Trainer(
        max_steps=args.max_steps,
        max_epochs=args.num_epochs,
        callbacks=[
            EMA(0.9999),
            ModelCheckpoint(
                dirpath=args.save_path,
                every_n_epochs=args.save_freq,
                save_top_k=args.top_k
            )],
        accelerator=args.accelerator,
        #devices=args.devices,
        logger=wandb_logger,
        log_every_n_steps=args.log_steps
    )

    trainer.fit(model)

    trainer.save_checkpoint(args.save_path)

In [4]:
train(args)

Files already downloaded and verified
Files already downloaded and verified
Is Time embed used ?  True


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gyulong1 (cs5340). Use `wandb login --relogin` to force relogin


/Users/yulong/Desktop/NUS/Year 3/Sem 2/CS5340/project/LDM4DD/utils/EMA.py:58: UserWarning: EMA has better performance when Apex is installed: https://github.com/NVIDIA/apex#installation.
  rank_zero_warn(


ValueError: Expected a parent